In [48]:
import numpy as np
import random
import math
import pandas as pd


In [49]:
def sigmoid(x):
    sig = 1 / (1 + math.exp(x))
    return sig

def sigmoid_derivate(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [50]:
class MLP(object):

    def __init__(self, L):
        self.Nl = len(L)
        self.L = L
        self.b = [np.random.randn(y,1) for y in L[1:]]
        self.w = [np.random.randn(y,x) for x, y in zip(L[:-1], L[1:])]

    def feedfoward(self, a):
        a = sigmoid(np.dot(self.w, a) + self.b)
        return a

    def training(self, Xd, Ne, Nb, eta, Xd_test=None):
        Xd = list(Xd)
        n = len(Xd)

        if Xd_test:
            Xd_test = list(Xd_test)
            n_test = len(Xd_test)

        for j in range(Ne):
            random.shuffle(Xd)

            mini_batches = [Xd[k:k+Nb] for k in range(0, n, Nb)]
            
            for mini_batch in mini_batches:
                self.update_perceptrons(mini_batch, eta)

            if Xd_test:
                print("Época {} finalizada: {} / {}".format(j,self.test(Xd_test), n_test))
            else:
                print("Época {} finalizada".format(j))


    
    def update_perceptrons(self, Xd, eta):
        grad_b = [np.zeros(b.shape) for b in self.b]
        grad_w = [np.zeros(w.shape) for w in self.w]

        for x, y in Xd:
            del_grad_b, del_grad_w = self.backpropagation(x, y)

            grad_b = [nb + dnb for nb,dnb in zip(grad_b, del_grad_b)]
            grad_w = [nw + dnw for nw,dnw in zip(grad_w, del_grad_w)]

        self.w = [w - (eta/len(Xd))*nw for w,nw in zip(self.w, grad_w)]
        self.b = [b - (eta/len(Xd))*nb for b,nb in zip(self.b, grad_b)]

    def backpropagation(self, x, y):
        grad_b = [np.zeros(b.shape) for b in self.b]
        grad_w = [np.zeros(w.shape) for w in self.w]

        activation = x

        activations = [x]

        zs = []

        for b, w in zip(self.b, self.w):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = (activations[-1]-y) * sigmoid_derivate(zs[-1])
        grad_b[-1] = delta
        grad_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.Nl):
            z = zs[-l]
            sp = sigmoid_derivate(z)
            delta = np.dot(self.w[-l+1].transpose(), delta) * sp
            grad_b[-l] = delta
            grad_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (grad_b, grad_w)

    def test(self, Xd_test):
        result = [(np.argmax(self.feedfoward(x)), y) for (x, y) in Xd_test]
        return sum(int( x == y) for (x, y) in Xd_test)


In [51]:
rede1 = MLP([3,2,1])

data_df = pd.read_csv("circles_and_squares.csv")
data_df_treino = data_df.iloc[0:800, :]
data_df_test = data_df.iloc[800:, :]

gabarito = data_df_test["400"]
gabarito = gabarito.to_numpy()
data_df_test = data_df_test.drop(columns=['400'])

Xd_test = data_df_test.to_numpy()

reference = data_df_treino["400"]
reference = reference.to_numpy()
data = data_df_treino.drop(columns=["400"])

Xd = data.to_numpy()

In [53]:
rede1.training((Xd, reference), 50, 100, 0.01,(Xd_test, gabarito))


ValueError: too many values to unpack (expected 2)